
# 深層学習における一般化

:numref: `chap_regression`と :numref: `chap_classification`では、線形モデルをトレーニング データに適合させることで回帰と分類の問題に取り組みました。どちらの場合も、観察されたトレーニング ラベルの尤度を最大化するパラメータを見つけるための実用的なアルゴリズムを提供しました。そして、各章の終わりに近づくにつれて、トレーニング データの適合は中間目標にすぎないことを思い出しました。私たちの本当の探求はずっと、同じ基礎となる母集団から抽出された新しい例であっても正確な予測を行うことができる*一般的なパターンを*発見することでした。機械学習の研究者は、最適化アルゴリズムの*利用者*です。場合によっては、新しい最適化アルゴリズムを開発する必要もあります。しかし、結局のところ、最適化は目的を達成するための手段にすぎません。本質的に、機械学習は統計分野であり、何らかの統計原理 (既知または未知) によって得られるモデルがトレーニング セットを超えて一般化される場合に限り、トレーニング損失を最適化したいと考えています。

明るい面としては、確率的勾配降下法によって訓練されたディープ ニューラル ネットワークが、コンピューター ビジョンに及ぶ無数の予測問題にわたって驚くほどうまく一般化できることがわかりました。自然言語処理;時系列データ。推薦システム。電子健康記録。タンパク質の折り畳み。ビデオゲームやボードゲームにおける値関数の近似。他にも無数のドメインがあります。欠点としては、最適化のストーリー (トレーニング データに適合できる理由) または一般化のストーリー (結果として得られたモデルがまだ見たことのない例に一般化される理由) についての簡単な説明を探している場合は、飲む。線形モデルを最適化するための手順とソリューションの統計的特性は両方とも包括的な理論体系によって十分に説明されていますが、深層学習に対する私たちの理解は、どちらの面でも依然として西部開拓時代に似ています。

ディープラーニングの理論と実践は両面で急速に進化しており、実践者が猛烈なペースで革新を続けているにもかかわらず、理論家は何が起こっているのかを説明するために新しい戦略を採用し、深いネットワークと一連の直観力をトレーニングするためのヒューリスティックの武器庫を構築しています。どの状況にどのテクニックを適用するかを決定するための指針となる民間知識。

現時点での TL;DR は、深層学習の理論は有望な攻撃方法と散在する興味深い結果を生み出してきたが、(i) なぜニューラル ネットワークを最適化できるのか、(ii) の両方についての包括的な説明にはまだ程遠いようだということです。 ) 勾配降下法で学習したモデルが、高次元のタスクであっても、どのようにしてうまく一般化できるのか。ただし、実際には、(i) が問題になることはめったにありません (すべてのトレーニング データに適合するパラメーターを常に見つけることができます)。したがって、一般化を理解することの方がはるかに大きな問題となります。一方で、一貫した科学理論という安心感がなくても、実践者は、実際に一般化できるモデルを作成するのに役立つ膨大なテクニックを開発してきました。深層学習における一般化という広大なテーマを正確に説明できる簡潔な要約はあり得ませんが、研究の全体的な状況は解決にはほど遠いですが、このセクションでは研究と実践の状況の広範な概要を提示したいと考えています。

## 過学習と正則化の再考

:citet: `wolpert1995no`による「フリー ランチなし」定理によれば、学習アルゴリズムは特定の分布のデータではより良く一般化しますが、他の分布ではより悪くなります。したがって、トレーニングセットが有限であるとすると、モデルは特定の仮定に依存します。人間レベルのパフォーマンスを達成するには、人間が世界についてどのように考えるかを反映する*帰納的バイアス*を特定することが役立つ可能性があります。このような誘導バイアスは、特定の特性を持つソリューションに対する優先順位を示します。たとえば、ディープ MLP には、より単純な関数を組み合わせて複雑な関数を構築するという帰納的バイアスがあります。

帰納的バイアスをエンコードした機械学習モデルを使用して、それらをトレーニングするアプローチは通常、次の 2 つのフェーズで構成されます。(i) トレーニング データを適合させる。 (ii) ホールドアウト データに基づいてモデルを評価することにより、*一般化誤差*(基礎となる母集団に関する真の誤差) を推定します。トレーニング データに対する適合とテスト データに対する適合の差は*汎化ギャップ*と呼ばれ、汎化ギャップが大きい場合、モデルがトレーニング データに*過剰適合している*と言われます。過剰適合の極端なケースでは、テスト誤差が依然として重大なままである場合でも、トレーニング データを正確に適合させる可能性があります。そして古典的な見方では、モデルが複雑すぎるため、特徴の数、学習される非ゼロパラメータの数、または定量化されたパラメータのサイズを縮小する必要があると解釈されます。 :numref: `sec_generalization_basics`のモデルの複雑さと損失のプロット (:numref: `fig_capacity_vs_error` ) を思い出してください。

しかし、ディープラーニングは、直観に反した方法でこの状況を複雑にします。まず、分類問題については、通常、私たちのモデルは、たとえ数百万のデータセットであっても、あらゆるトレーニング例に完全に適合するのに十分な表現力を持っています :cite: `zhang2021understanding` 。古典的な図では、この設定はモデルの複雑さ軸の右端にあり、汎化誤差の改善は、モデル クラスの複雑さを軽減するか、またはこれはペナルティであり、パラメーターが取り得る値のセットを厳しく制限します。しかし、そこから事態がおかしくなり始めます。

奇妙なことに、多くの深層学習タスク (画像認識やテキスト分類など) では、通常、モデル アーキテクチャの中から選択しており、そのすべてが任意に低いトレーニング損失 (およびトレーニング エラーゼロ) を達成できます。検討中のすべてのモデルはトレーニング誤差ゼロを達成しているため、*さらなる利益を得るための唯一の手段は、過学習を減らすことです*。さらに奇妙なことに、トレーニング データを完全にフィッティングしたにもかかわらず、モデルの*表現力をさらに*高めることによって、実際には*汎化誤差をさらに削減できる*場合がよくあります。たとえば、レイヤー、ノードを追加したり、より多くのエポックのトレーニングを行ったりすることができます。さらに奇妙なことに、汎化ギャップをモデルの*複雑さ*に関連付けるパターン (ネットワークの深さや幅などで捉えられる) は単調ではない場合があり、複雑さが増すと最初は害を及ぼすものの、その後はいわゆる「問題解決」に役立ちます。 「二重降下」パターン :cite: `nakkiran2021deep` 。したがって、深層学習の実践者はさまざまなトリックを所有しており、そのいくつかはモデルを何らかの方法で制限するように見えるものや、モデルをさらに表現力豊かにするように見えるものもあり、それらはすべてある意味、過学習を軽減するために適用されます。

事態をさらに複雑にするのは、古典的な学習理論によって提供される保証は古典的なモデルであっても保守的である可能性がある一方で、そもそもディープ ニューラル ネットワークがなぜ一般化するのかを説明するのに無力であるように見えることです。ディープ ニューラル ネットワークは、大規模なデータセットであっても任意のラベルを当てはめることができるため、$\ell_2$ 正則化などのよく知られた手法を使用しているにもかかわらず、従来の複雑さに基づく一般化限界 (仮説の VC 次元や Rademacher 複雑さに基づくものなど)クラスでは、ニューラル ネットワークが一般化する理由を説明できません。

## ノンパラメトリクスからのインスピレーション

ディープラーニングに初めて取り組むと、深層学習をパラメトリック モデルとして考えたくなります。結局のところ、モデルには何百万ものパラメータが*あり*ます。モデルを更新すると、そのパラメーターも更新されます。モデルを保存すると、そのパラメータがディスクに書き込まれます。しかし、数学とコンピューター サイエンスには、直観に反する視点の変化や、一見異なる問題に見える驚くべき同型性の問題がたくさんあります。ニューラル ネットワークには明らかにパラメーターが*あります*が、ある意味では、ノンパラメトリック モデルのように動作すると考える方がより有益な場合があります。では、正確には何がモデルをノンパラメトリックにしているのでしょうか?この名前はさまざまなアプローチをカバーしていますが、共通のテーマの 1 つは、ノンパラメトリック手法は利用可能なデータの量が増えるにつれて複雑さのレベルが増す傾向にあるということです。

おそらく、ノンパラメトリック モデルの最も単純な例は、$k$-最近傍アルゴリズムです (後で、 :numref: `sec_attention-pooling`などの他のノンパラメトリック モデルについて説明します)。ここで、学習者はトレーニング時にデータセットを単に記憶するだけです。次に、予測時に新しい点 $\mathbf{x}$ に直面すると、学習者は $k$ の最近傍点 ($k$ 点 $\mathbf{x}_i&#39;$ は距離 $ d(\mathbf{x}, \mathbf{x}_i&#39;)$)。 $k=1$ の場合、このアルゴリズムは 1 最近傍アルゴリズムと呼ばれ、アルゴリズムは常にトレーニング誤差 0 を達成します。ただし、アルゴリズムが一般化しないという意味ではありません。実際、いくつかの穏やかな条件下では、1 最近傍アルゴリズムは一貫している (最終的には最適な予測子に収束する) ことがわかります。

 1 つの最近傍では、データを特徴付けるために何らかの距離関数 $d$ を指定するか、または同等のベクトル値の基底関数 $\phi(\mathbf{x})$ を指定する必要があることに注意してください。距離計量をどのように選択しても、学習誤差は 0 になり、最終的には最適な予測子に到達しますが、異なる距離計量 $d$ は異なる帰納的バイアスをエンコードし、有限量の利用可能なデータでは異なる予測子が得られます。距離計量 $d$ のさまざまな選択は、基礎となるパターンに関するさまざまな仮定を表し、さまざまな予測子のパフォーマンスは、その仮定が観察されたデータとどの程度互換性があるかによって異なります。

ある意味、ニューラル ネットワークは過剰にパラメータ化されており、トレーニング データに適合させるために必要なパラメーターよりも多くのパラメーターを所有しているため、トレーニング データ*を補間する*(完全に適合させる) 傾向があり、そのため、ある意味ではノンパラメトリック モデルに近い動作をします。最近の理論研究では、大規模なニューラル ネットワークとノンパラメトリック手法、特にカーネル手法との間に深いつながりが確立されています。特に、:citet: `Jacot.Grabriel.Hongler.2018`は、ランダムに初期化された重みを持つ多層パーセプトロンが無限に広がるにつれて、カーネル関数の特定の選択 (本質的に、距離関数)、彼らはこれをニューラル タンジェント カーネルと呼んでいます。現在のニューラル タンジェント カーネル モデルは、最新のディープ ネットワークの動作を完全には説明していない可能性がありますが、分析ツールとしての成功は、過剰にパラメータ化されたディープ ネットワークの動作を理解するためのノンパラメトリック モデリングの有用性を強調しています。

## 早期停止

ディープ ニューラル ネットワークは、ラベルが間違ってまたはランダムに割り当てられた場合でも、任意のラベルを当てはめることができますが :cite: `zhang2021understanding` 、この能力はトレーニングを何度も反復することで初めて現れます。新しい研究結果 :cite: `Rolnick.Veit.Belongie.Shavit.2017`は、ラベル ノイズの設定において、ニューラル ネットワークが最初にきれいにラベル付けされたデータを適合させ、その後にのみ誤ってラベル付けされたデータを補間する傾向があることを明らかにしました。さらに、この現象は一般化の保証に直接変換されることが確立されています。モデルがトレーニング セットに含まれるランダムにラベル付けされた例ではなく、きれいにラベル付けされたデータに適合するときはいつでも、実際には一般化されています。 :cite: `Garg.Balakrishnan.Kolter.Lipton.2021` · 。

これらの発見を総合すると、ディープ ニューラル ネットワークを正規化するための古典的な手法である*早期停止を*動機付けるのに役立ちます。ここでは、重みの値を直接制約するのではなく、トレーニングのエポック数を制約します。停止基準を決定する最も一般的な方法は、トレーニング全体を通じて検証エラーを監視し (通常は各エポック後に 1 回チェックする)、検証エラーが特定の数値に対して少量 $\epsilon$ 以上減少しなかった場合にトレーニングを中止することです。エポックの。これは*忍耐基準*と呼ばれることもあります。ノイズの多いラベルの設定では、より良い一般化につながる可能性があることに加えて、早期に停止することのもう 1 つの利点は、時間が節約されることです。忍耐基準が満たされると、トレーニングを終了できます。 8 個以上の GPU で同時に数日間のトレーニングを必要とする可能性がある大規模なモデルの場合、適切に調整された早期停止により、研究者の時間を数日節約し、雇用主は数千ドルを節約できます。

特に、ラベルノイズがなく、データセットが*実現可能で*ある場合（クラスが真に分離可能、たとえば猫と犬を区別できる場合）、早期に停止しても汎化の大幅な改善につながらない傾向があります。一方、ラベルノイズやラベルに固有のばらつきがある場合（例：患者間の死亡率の予測）、早期に中止することが重要です。ノイズの多いデータを補間するまでモデルをトレーニングすることは通常、悪い考えです。

## ディープネットワーク向けの古典的な正則化手法

:numref: `chap_regression`では、モデルの複雑さを制限するためのいくつかの古典的な正則化手法について説明しました。特に、:numref: `sec_weight_decay` 、重み減衰と呼ばれる方法を導入しました。これは、損失関数に正則化項を追加して重みの大きな値にペナルティを与えることで構成されます。どの重みノルムにペナルティが課されるかに応じて、この手法はリッジ正則化 ($\ell_2$ ペナルティの場合) またはラッソ正則化 ($\ell_1$ ペナルティの場合) として知られています。これらの正則化子の古典的な分析では、モデルが任意のラベルに適合するのを防ぐために、重みが取り得る値を十分に制限すると考えられます。

深層学習の実装では、重み減衰は依然として人気のあるツールです。ただし、研究者らは、$\ell_2$ 正則化の一般的な強みは、ネットワークによるデータの補間を防ぐには不十分であると指摘しています:cite: `zhang2021understanding`したがって、正則化として解釈された場合の利点は、早期停止基準と組み合わせてのみ意味をなす可能性があります。早期に停止しないと、層の数やノードの数 (深層学習の場合)、または距離メトリック (1 近傍の場合) と同様に、これらの方法がより良い一般化につながる可能性があります。ニューラル ネットワークではなく、関心のあるデータセットで見つかったパターンとより互換性のある誘導バイアスを何らかの形でエンコードしているためです。したがって、たとえその有効性の理論的根拠が根本的に異なる場合でも、古典的な正則化器は深層学習の実装において依然として人気があります。

注目すべきことに、深層学習の研究者は、モデル入力にノイズを追加するなど、古典的な正則化のコンテキストで最初に普及した手法も構築しています。次のセクションでは、有名なドロップアウト手法 (:citet: `Srivastava.Hinton.Krizhevsky.ea.2014`によって発明) を紹介します。この手法は、その有効性の理論的根拠が同様に謎のままであるにもかかわらず、深層学習の主流となっています。

## まとめ

例よりもパラメーターが少ない傾向にある古典的な線形モデルとは異なり、ディープ ネットワークはパラメーターが過剰になる傾向があり、ほとんどのタスクではトレーニング セットを完全に適合させることができます。この*補間方式は、*多くの堅固な直感に疑問を投げかけます。機能的には、ニューラル ネットワークはパラメトリック モデルのように見えます。しかし、それらをノンパラメトリック モデルとして考えると、より信頼できる直観の源となる場合があります。多くの場合、検討中のすべてのディープ ネットワークはすべてのトレーニング ラベルに適合できるため、ほぼすべての利益は過適合を軽減する (*汎化ギャップ*を埋める) ことによってもたらされる必要があります。逆説的ですが、一般化ギャップを減らす介入は、モデルの複雑さを増大させるように見えることもあれば、複雑さを軽減するように見えることもあります。しかし、これらの方法では、古典理論が深いネットワークの一般化を説明できるほど複雑さが軽減されることはめったになく、*なぜ特定の選択が一般化の向上につながるのかは、*多くの優秀な研究者の協力的な努力にもかかわらず、大部分が未解決の大きな疑問のままです。

## 演習
1. 従来の複雑さに基づく測定では、どのような意味でディープ ニューラル ネットワークの一般化を説明できないのでしょうか?
1. *早期停止が*正則化手法とみなされるのはなぜですか?
1. 研究者は通常、停止基準をどのように決定しますか?
1. 早期に中止することで一般化が大幅に改善される場合を区別する重要な要素は何でしょうか?
1. 一般論を超えて、早期に中止することのもう 1 つの利点について説明します。

[ディスカッション](https://discuss.d2l.ai/t/7473)
